This notebook aims on cross-compare and double check accuracy and precision of topic modeling label. </p>
Main steps in this notebook:</p>
1) pull up representative speeches labled by each model about environment;</p>
2) cross-check the results from each model: do they share a lot common? Do all models label the same group of speech as environment-relevant? </p>
3) Mannual check precision and accuracy for randomly selected speeches.</p>
4) Explore collocated-words in interested speeches.</p>
5) merge speeches with speech information (like speakers information) and output environment speeches and data for peer lab member to further investigate

In [ ]:
# code to find topic with Standford keywords
# environment: 3/ 520
# sd 406: https://www.sanders.senate.gov/press-releases/sanders-boxer-to-introduce-major-climate-change-legislation-2/
envir_topic_bert = []
for topicIndex in topics:
    for pair in topics[topicIndex]:
        for i in range(len(pair)):
            for word in environment_keyWord:
                if word == pair[i]:
                    envir_topic_bert.append(topicIndex)
                    print(topicIndex)  # each key words
            

In [1]:
import pandas as pd
fullData = pd.read_csv('test50_labled.csv')
bert_kw_List = [3,2761, 4476, 2343, 4294, 7408, 43,110,115,595,672,849,1311,2643,2805,2855,3184,3878,4148,4690,4821,4887,5892,6154,6159,6228,6532,6675,6820]
lda_kw_List = [9]
nmf_kw_List = [18,19,25]

In [18]:
new_data = pd.read_csv('test50.csv')
fullData = pd.merge(fullData, new_data[['speech_id', 'date']], on='speech_id', how='left')
fullData = pd.merge(fullData, new_data[['speech_id', 'Year']], on='speech_id', how='left')

In [19]:
import numpy as np
import csv
import glob
import nltk
from nltk.corpus import wordnet as wn

import nltk
nltk.download('wordnet')

from collections import Counter
fullDataBack =fullData
fullData['speech'] = fullData['speech'].str.split().dropna()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
def lemmatize_words(word_list):
    if isinstance(word_list, list):
        return [wn.morphy(word) for word in word_list]
    else:
        return []

# Apply the lemmatization function to the 'speech' column
fullData['lemmas'] = fullData['speech'].apply(lemmatize_words)

In [35]:
def convert_to_list(s):
    try:
        # Try to evaluate the string as a Python literal
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        # If there's an error, return an empty list
        return []

# Apply the conversion function to the 'speech' column
fullData['speech'] = fullData['speech'].apply(convert_to_list)

# Filter out rows containing the word 'energy'
filtered_data = fullData[fullData['speech'].apply(lambda x: 'oil' in x)]


In [36]:
filtered_data

""


In [33]:
#fullData = fullData[fullData["speech"].contains('energy')]
#fullData['lemmas'] = [[wn.morphy(word) for word in row] for row in fullData['speech']]
fullData=filtered_data
fullData['lemmas'] = [[word for word in row if word is not None] for row in fullData['lemmas']] # for each word of each row in 'lemmas', keep anything that isn't 'None' type.
#fullData['lemmas'] = [' '.join(row) for row in fullData['lemmas']] # glue the individual list of lemmas back into one string per speech

top_words_per_year_lemmatized = {}

for year in fullData['Year'].unique():
    yearly_lemmas = []
    for lemmas in fullData[fullData['Year'] == year]['lemmas']:
        yearly_lemmas.extend(lemmas)

    # Calculate frequency
    lemma_counts = Counter(yearly_lemmas)

    # Get the 10 most common lemmas
    top_lemmas = lemma_counts.most_common(10)
    top_words_per_year_lemmatized[year] = top_lemmas

top_words_per_year_lemmatized

{}

In [ ]:
fullData['speech_id'].describe()
#1060000000 to 1140000000
filtered_data = fullData[(fullData['speech_id'] >= 1060000000) & (fullData['speech_id'] < 1070000000)]['speech']


In [ ]:
year1_context_count = fullData["speech"].str.split().explode().dropna().value_counts()
year1_context_count[:20]

In [ ]:
pd.set_option('display.max_colwidth', None)
fullData[fullData['Topic']==2343]['speech'].head(3)

In [ ]:
# print out each topic keywords with an example
pd.set_option('display.max_colwidth', None)
for num in bert_kw_List:
    print(num)
    print(fullData[fullData['Topic']==num]['Representation'].head(1))
    print(fullData[fullData['Topic']==num]['speech'].head(1))

In [ ]:
bert_sp_List = set()
lda_sp_List = set()
nmf_sp_List = set()
check_id = 0

for i in range(len(fullData)):
    check_id=check_id+1
    id=fullData['speech_id'][i]
    for bkw in bert_kw_List:
        if fullData['Topic'][i] == bkw:
            bert_sp_List.add(id)
    for lkw in lda_kw_List:
        if fullData['Most_Dominant_Topic_LDA'][i] == lkw:
            lda_sp_List.add(id)
    for nkw in nmf_kw_List:
        if fullData['Most_Dominant_Topic_NMF'][i] == nkw:
            nmf_sp_List.add(id)

#bert_sp_List = set(bert_sp_List)
#lda_sp_List = set(lda_sp_List)
#nmf_sp_List = set(nmf_sp_List)

collocated words for top keywords
before and after 2008 and 2001
    some readings
check out the pattern/ trend for minor topic ( without topic 3)
sentimental analysis
    tidytext


In [2]:
bert_kw_List = [3,2761, 4476, 2343, 7408, 43,115,595,672,1311,2643,2805,2855,3184,4148,4690,4821,4887,5892,6154,6159,6532,6675,6820]
#df = pd.read_csv('gpt4_5000_result.csv')


bert_sp_List = set()
check_id = 0

for i in range(len(fullData)):
    id=fullData['speech_id'][i]
    for bkw in bert_kw_List:
        if fullData['Topic'][i] == bkw:
            check_id=check_id+1
            bert_sp_List.add(id)

#bert_sp_List = set(bert_sp_List)
print(check_id)
common_speech_ids = set(df['speech_id']).intersection(bert_sp_List)
print("yes")
print(len(common_speech_ids))


# Convert set to a Pandas DataFrame
#df = pd.DataFrame({'set_column': [bert_sp_List]})

# Specify the CSV file path
#csv_file_path = 'random50EnvironmentSpeechid.csv'

# Write to CSV file
#df.to_csv(csv_file_path, index=False)



7810
yes
72


In [19]:
# save whatever into pickle
pickle_file_path = 'bert_environ.pkl'

# Open the pickle file for writing in binary mode ('wb')
with open(pickle_file_path, 'wb') as file:
    # Dump the set into the pickle file
    pickle.dump(common_speech_ids, file)

In [3]:
# read Louis pickle and take out mannual annotated "yes"
import pickle

# Specify the path to your pickle file
pickle_file_path = 'manualAnnotation.pkl'

# Open the pickle file for reading in binary mode ('rb')
with open(pickle_file_path, 'rb') as file:
    # Load the data from the pickle file
    result35 = pickle.load(file)
# mannual annotate
keys_with_y_value = {key for key, value in result35.items() if value == 'y'}

In [18]:
# compare set of "yes" from various models
#87 vs 72: 22 in common
common_speech_ids_bert_35 = keys_with_y_value.intersection(common_speech_ids)
print(len(common_speech_ids_bert_35))

22


In [15]:
# Convert set to Pandas DataFrame
import random
sampled_items = random.sample(sorted(bert_sp_List), 100)

df = pd.DataFrame({'set_column': [sampled_items]})

# Explode the set into separate rows
df_exploded = df['set_column'].explode().reset_index(drop=True).to_frame()

# Specify the CSV file path
csv_file_path = 'random50EnvironmentSpeechid_100.csv'

# Write to CSV file
df_exploded.to_csv(csv_file_path, index=False, header=['set_column'])

In [25]:
# cross-check environment topic from bert
fullData = pd.read_csv('test50_labled.csv')
bert_kw_List = [3,2761, 4476, 2343, 4294, 7408, 43,110,115,595,672,849,1311,2643,2805,2855,3184,3878,4148,4690,4821,4887,5892,6154,6159,6228,6532,6675,6820]

filtered_data = fullData[
    fullData.apply(lambda row: row['Representative_document'] and row['Topic'] in bert_kw_List, axis=1)
]

In [33]:
# open json for bertopic keyword and topic dictionary
import json
# Specify the path to your JSON file
json_file_path = 'Bertopic_50_dic.json'

# Open the JSON file for reading
with open(json_file_path, 'r') as file:
    # Load the JSON data from the file
    bertKeyWordic = json.load(file)


#### Note about Bertopic result
bert_kw_List = [3,2761, 4476, 2343, 7408, 43,115,595,672,1311,2643,2805,2855,3184,4148,4690,4821,4887,5892,6154,6159,6532,6675,6820]

previously wrong from standford keyword: 110( the word "object"), 4294('na'), 6228('include location Penn'), 849( include California), 3878(texas)

In [ ]:
bert_kw_List = [3,2761, 4476, 2343, 7408, 43,115,595,672,1311,2643,2805,2855,3184,4148,4690,4821,4887,5892,6154,6159,6532,6675,6820]
filtered_data_only_correct_en = fullData[
    fullData.apply(lambda row: row['Topic'] in bert_kw_List, axis=1)
]
filtered_data_only_correct_en.to_csv("Environment")

In [ ]:
[['speech','chamber','date','speaker','first_name','last_name','state','gender','char_count','word_count']]

In [ ]:
fullData

In [43]:
# all demographic information for speeches
# with full demo data
fullData = pd.read_csv('fullData2000.csv')
# join topic and keywords
lables = pd.read_csv('test50_labled.csv')
label_fullSpeech_demo = pd.merge(lables[['Topic', 'Representation','Representative_document','speech_id']], 
                                 fullData[['speech_id','speech','chamber','date','speaker','first_name','last_name','state','gender','char_count','word_count']],
                                 on='speech_id', how='left')

# filter and only keep environment relevant topics
bert_kw_List = [3,2761, 4476, 2343, 7408, 43,115,595,672,1311,2643,2805,2855,3184,4148,4690,4821,4887,5892,6154,6159,6532,6675,6820]
filtered_data_demo = label_fullSpeech_demo[label_fullSpeech_demo.apply(lambda row: row['Topic'] in bert_kw_List, axis=1)]
#save to csv
filtered_data_demo.to_csv("EnvironmentSpeech_WithInfo.csv")

In [ ]:
filtered_data
fullData = pd.read_csv('fullData2000.csv')

result_df = pd.merge(filtered_data, fullData[['speech_id', 'speech']], on='speech_id', how='left')
